In [2]:
sqlDF = spark.sql("SELECT * FROM transactions")
display(sqlDF)



In [3]:
sqlDF = spark.sql("SELECT msno, MAX(transaction_date) as transaction_date, first(payment_method_id) as payment_method_id, AVG(payment_plan_days) as payment_plan_days,  first(plan_list_price) as plan_list_price, first(actual_amount_paid) as actual_amount_paid, first(is_auto_renew) as is_auto_renew, MAX(membership_expire_date) as membership_expire_date, first(is_cancel) as is_cancel FROM transactions GROUP BY msno ")
display(sqlDF)


#most of the data is catagorical in this file

In [4]:
sqlDF2 = spark.sql("SELECT * FROM user_logs_full")
display(sqlDF2)


In [5]:
sqlDF2 = spark.sql("SELECT msno, SUM(num_25) AS num_25_sum, SUM(num_50) AS num_50_sum, SUM(num_75) AS num_75_sum, SUM(num_985) AS num_985_sum, SUM(num_100) AS num_100_sum, SUM(num_unq) AS num_unq_sum, AVG(total_secs) AS total_secs_avg FROM user_logs_full GROUP BY msno ")
display(sqlDF2)



In [6]:
#this is the limiting dataset
sqlDF2.count()


In [7]:
sqlDF3 = spark.sql("SELECT * FROM train")
display(sqlDF3)

In [8]:
final = sqlDF.join(sqlDF2, 'msno', "outer")
final2 = final.join(sqlDF3, 'msno', "outer")
final3 = final2.na.drop(subset=["num_75_sum"])
final4 = final3.na.drop(subset=["is_churn"])




In [9]:
final4.count()

In [10]:
display(final4)

In [11]:
from pyspark.ml.feature import RFormula

rf = RFormula(formula="~ payment_method_id +  payment_plan_days + plan_list_price - 1")
final_df_rf = rf.fit(sqlDF).transform(sqlDF)
final_df_rf.select("features").show(4, False)


In [12]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=["payment_method_id", "plan_list_price", "actual_amount_paid"], outputCol="features")
final_df = assembler.transform(sqlDF)
final_df.select("features").show(4, False)



